In [141]:
import networkx as nx
import igraph as ig
import louvain
from collections import OrderedDict

##### Create a total graph and alliance graph
NB: is necessary run this code only one time (you probably have already the net in the dataset folder); i hope wont be necessary come back on this

In [5]:
# create a multigraph for all nets type
messageGraph = nx.MultiDiGraph()
tradeGraph = nx.MultiDiGraph()
attackGraph = nx.MultiDiGraph()

# number of graph (30 in our case)
nGraph = 30

# for every day is needed create total and alliance graph
for i in range(0, nGraph):
    print("Graph number: ", (i + 1))
    
    # for every day is needed reinitialize the "real" graphs (graph with edge label)
    realAttack = nx.MultiDiGraph()
    realMessage = nx.MultiDiGraph()
    realTrade = nx.MultiDiGraph()
    
    # for every day is needed reinitialize the alliance graph and total graph
    allianceGraph = nx.MultiDiGraph()
    totalGraph = nx.MultiDiGraph()
    
    # read from Dataset/Graphml-File the correct file based on type and day
    attackGraph = nx.read_graphml("../Dataset/Graphml-File/Attacks-Network-Graphml/attacks-timestamped-2009-12-"+str(i+1)+".graphml")
    messageGraph = nx.read_graphml("../Dataset/Graphml-File/Messages-Network-Graphml/messages-timestamped-2009-12-"+str(i+1)+".graphml")
    tradeGraph = nx.read_graphml("../Dataset/Graphml-File/Trades-Network-Graphml/trades-timestamped-2009-12-"+str(i+1)+".graphml")
    
    # copy xxxxGraph in realxxxx adding label to edges; keep attentio about key, it must be unique (we use old key + 
    # letter for define edge tipology) 
    for itr in messageGraph.edges_iter(None, True, True):
        realMessage.add_edge(int(itr[0]),int(itr[1]), key=str(itr[2])+"m", Label= "Message", time = itr[3]['edgetime'])
    
    for itr in tradeGraph.edges_iter(None, True, True):
        realTrade.add_edge(int(itr[0]),int(itr[1]), key=str(itr[2])+"t", Label= "Trade", time = itr[3]['edgetime'])

    for itr in attackGraph.edges_iter(None, True, True):
        realAttack.add_edge(int(itr[0]),int(itr[1]), key=str(itr[2])+"a", Label= "Attack", time = itr[3]['edgetime'])
    
    # compose (a union) in one graph the needed graphs 
    allianceGraph = nx.compose(realMessage, realTrade)
    totalGraph = nx.compose(allianceGraph, realAttack)
    # save total graph and alliance graph in the correct file
    nx.write_graphml(totalGraph, "../Dataset/Graphml-File/Total-Network-Graphml/total-2009-12-"+str(i+1)+".graphml")
    nx.write_graphml(realMessage, "../Dataset/Graphml-File/Alliance-Network-Graphml/alliance-2009-12-"+str(i+1)+".graphml")

    # print for check, i checked that sum of graphs is the equal to alliance graph and total graph (according to the case)
    # but i think is a better that YOU MAKE THE SAME; ONE MORE CONTROL IS BETTER!
    print("ATTACK GRAPH: ", nx.info(attackGraph))
    print("MESSAGE GRAPH: ", nx.info(messageGraph))
    print("TRADE GRAPH: ", nx.info(tradeGraph))
    print("ALLIANCE GRAPH: ", nx.info(allianceGraph))
    print("TOTAL GRAPH: ", nx.info(totalGraph))
    print("") 

Graph number:  1
ATTACK GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 2807
Number of edges: 32997
Average in degree:  11.7553
Average out degree:  11.7553
MESSAGE GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 1963
Number of edges: 16346
Average in degree:   8.3271
Average out degree:   8.3271
TRADE GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 1336
Number of edges: 9390
Average in degree:   7.0284
Average out degree:   7.0284
ALLIANCE GRAPH:  Name: compose( ,  )
Type: MultiDiGraph
Number of nodes: 2221
Number of edges: 25736
Average in degree:  11.5876
Average out degree:  11.5876
TOTAL GRAPH:  Name: compose( ,  )
Type: MultiDiGraph
Number of nodes: 3757
Number of edges: 58733
Average in degree:  15.6330
Average out degree:  15.6330

Graph number:  2
ATTACK GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 2802
Number of edges: 32782
Average in degree:  11.6995
Average out degree:  11.6995
MESSAGE GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 2100
Number of edges: 

ATTACK GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 2082
Number of edges: 21628
Average in degree:  10.3881
Average out degree:  10.3881
MESSAGE GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 1833
Number of edges: 14447
Average in degree:   7.8816
Average out degree:   7.8816
TRADE GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 1145
Number of edges: 9310
Average in degree:   8.1310
Average out degree:   8.1310
ALLIANCE GRAPH:  Name: compose( ,  )
Type: MultiDiGraph
Number of nodes: 1998
Number of edges: 23757
Average in degree:  11.8904
Average out degree:  11.8904
TOTAL GRAPH:  Name: compose( ,  )
Type: MultiDiGraph
Number of nodes: 2967
Number of edges: 45385
Average in degree:  15.2966
Average out degree:  15.2966

Graph number:  13
ATTACK GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 2066
Number of edges: 20675
Average in degree:  10.0073
Average out degree:  10.0073
MESSAGE GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 1786
Number of edges: 14597
Average in

ATTACK GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 1732
Number of edges: 16477
Average in degree:   9.5133
Average out degree:   9.5133
MESSAGE GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 1632
Number of edges: 13178
Average in degree:   8.0748
Average out degree:   8.0748
TRADE GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 1013
Number of edges: 8062
Average in degree:   7.9585
Average out degree:   7.9585
ALLIANCE GRAPH:  Name: compose( ,  )
Type: MultiDiGraph
Number of nodes: 1765
Number of edges: 21240
Average in degree:  12.0340
Average out degree:  12.0340
TOTAL GRAPH:  Name: compose( ,  )
Type: MultiDiGraph
Number of nodes: 2610
Number of edges: 37717
Average in degree:  14.4510
Average out degree:  14.4510

Graph number:  24
ATTACK GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 1716
Number of edges: 16467
Average in degree:   9.5962
Average out degree:   9.5962
MESSAGE GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 1618
Number of edges: 11773
Average in

##### Creation of our structure for maintain all nets
Create a dictionary of lists, the key rappresents the day (start 0 end 29), the list is a list of nets about the specific day with this order: (attackGraph, messageGraph, TradeGraph, allianceGraph, totalGraph)

In [3]:
# our structure 
allNetwork = dict()

nGraph = 30

# for structure graphs inserting
attackGraph = ig.Graph()
messageGraph = ig.Graph()
tradeGraph = ig.Graph()
totalGraph = ig.Graph()
allianceGraph = ig.Graph()

for i in range(0, nGraph):
    print("Day: ", (i + 1))
    
    # read all needed graphs
    attackGraph = attackGraph.Read_GraphML("../Dataset/Graphml-File/Attacks-Network-Graphml/attacks-timestamped-2009-12-"+str(i+1)+".graphml")
    messageGraph = messageGraph.Read_GraphML("../Dataset/Graphml-File/Messages-Network-Graphml/messages-timestamped-2009-12-"+str(i+1)+".graphml")
    tradeGraph = tradeGraph.Read_GraphML("../Dataset/Graphml-File/Trades-Network-Graphml/trades-timestamped-2009-12-"+str(i+1)+".graphml")
    allianceGraph = allianceGraph.Read_GraphML("../Dataset/Graphml-File/Alliance-Network-Graphml/alliance-2009-12-"+str(i+1)+".graphml")
    totalGraph = totalGraph.Read_GraphML("../Dataset/Graphml-File/Total-Network-Graphml/total-2009-12-"+str(i+1)+".graphml")
    
    # put as a list in a dictionary 
    allNetwork[i] = (attackGraph, messageGraph, tradeGraph, allianceGraph, totalGraph) 

Day:  1
Day:  2
Day:  3
Day:  4
Day:  5
Day:  6
Day:  7
Day:  8
Day:  9
Day:  10
Day:  11
Day:  12
Day:  13
Day:  14
Day:  15
Day:  16
Day:  17
Day:  18
Day:  19
Day:  20
Day:  21
Day:  22
Day:  23
Day:  24
Day:  25
Day:  26
Day:  27
Day:  28
Day:  29
Day:  30


### This below cell will be use for any kind of check about nets, variables, indexes; for this reason no one comments are required

In [194]:
for i in range(0, len(allNetwork)):
    for vertex in allNetwork[i][4].vs:
        if vertex['id'] == '2789':
            print("{:>20}{:>2}".format("Present in the day:", str(i + 1)))
print("")     
for i in range(0, len(allNetwork)):
    print("DAY: "+str(i+1))
    for edge in allNetwork[i][4].es:
        if allNetwork[i][4].vs[edge.tuple[0]]['id'] == '2789':
            print(edge)
print("")
for i in range(0, len(allNetwork)):
    print("DAY: "+str(i+1))
    for edge in allNetwork[i][4].es:
        if allNetwork[i][4].vs[edge.tuple[1]]['id'] == '2789':
            print(edge)
        

# 11850

 Present in the day: 1
 Present in the day: 2
 Present in the day: 3
 Present in the day: 4
 Present in the day: 5
 Present in the day: 6
 Present in the day: 7
 Present in the day: 8
 Present in the day: 9
 Present in the day:10
 Present in the day:11
 Present in the day:12
 Present in the day:13
 Present in the day:14
 Present in the day:15
 Present in the day:16
 Present in the day:17
 Present in the day:18
 Present in the day:19
 Present in the day:20
 Present in the day:21
 Present in the day:22
 Present in the day:24
 Present in the day:25
 Present in the day:26
 Present in the day:28

DAY: 1
igraph.Edge(<igraph.Graph object at 0x114c406d8>, 56098, {'key': '0t', 'time': 1259690340.0, 'Label': 'Trade'})
igraph.Edge(<igraph.Graph object at 0x114c406d8>, 56099, {'key': '1t', 'time': 1259701108.0, 'Label': 'Trade'})
igraph.Edge(<igraph.Graph object at 0x114c406d8>, 56100, {'key': '2t', 'time': 1259684008.0, 'Label': 'Trade'})
igraph.Edge(<igraph.Graph object at 0x114c406d8>, 56101, {

DAY: 12
igraph.Edge(<igraph.Graph object at 0x11ea43d68>, 24098, {'key': '0t', 'time': 1260603854.0, 'Label': 'Trade'})
igraph.Edge(<igraph.Graph object at 0x11ea43d68>, 24099, {'key': '1t', 'time': 1260678762.0, 'Label': 'Trade'})
igraph.Edge(<igraph.Graph object at 0x11ea43d68>, 24100, {'key': '2t', 'time': 1260651542.0, 'Label': 'Trade'})
igraph.Edge(<igraph.Graph object at 0x11ea43d68>, 24101, {'key': '3t', 'time': 1260631069.0, 'Label': 'Trade'})
DAY: 13
igraph.Edge(<igraph.Graph object at 0x121915318>, 42511, {'key': '0t', 'time': 1260697511.0, 'Label': 'Trade'})
igraph.Edge(<igraph.Graph object at 0x121915318>, 42512, {'key': '1t', 'time': 1260735166.0, 'Label': 'Trade'})
igraph.Edge(<igraph.Graph object at 0x121915318>, 42513, {'key': '2t', 'time': 1260747401.0, 'Label': 'Trade'})
igraph.Edge(<igraph.Graph object at 0x121915318>, 42514, {'key': '3t', 'time': 1260724267.0, 'Label': 'Trade'})
DAY: 14
igraph.Edge(<igraph.Graph object at 0x1219157c8>, 43034, {'key': '0t', 'time': 1

# START ANALYSIS !!!!

### Users number trend in the 30 days

In [4]:
def userNumberTrend(allNetwork):
    nUsers = [0]*len(allNetwork)
    for i in range(0, len(allNetwork)):
        nUsers[i] = allNetwork[i][4].vcount()

    print("Users in the 30 days: ", nUsers)
    print("")
    print("{:>53}{:>7}.".format("Average users number in the 30 days is: ", sum(nUsers)/len(nUsers)))
    print("{:>53}{:>7}.".format("The maximum number of users was recorded on the day: ", nUsers.index(max(nUsers))+1))
    print("")

### Edges number trend in the 30 days

In [5]:
def edgeNumberTrend(allNetwork):  
    nAttack = [0]*len(allNetwork)
    nTrade = [0]*len(allNetwork)
    nMessage = [0]*len(allNetwork)

    for i in range(0, nGraph):
        for edge in allNetwork[i][4].es:
            if edge['Label'] == "Attack":
                nAttack[i] += 1
            elif edge['Label'] == "Trade":
                nTrade[i] += 1
            elif edge['Label'] == "Message":
                nMessage[i] += 1
            else:
                print("This is a problem...")

    print("{:>33}".format("Number of attacks in the 30 days:"), end = " ")
    print(nAttack)
    print("")
    print("{:>32}".format("Number of trades in the 30 days:"), end = " ")
    print(nTrade)
    print("")
    print("{:>34}".format("Number of messages in the 30 days:"), end = " ")
    print(nMessage)
    print("")
    print("{:>40}{:>9}".format("Average number of attacks in 30 days:", sum(nAttack)/len(nAttack)))
    print("{:>40}{:>9}".format("Average number of trades a in 30 days:", round(sum(nTrade)/len(nTrade), 1)))
    print("{:>40}{:>9}".format("Average number of messages a in 30 days:", round(sum(nMessage)/len(nMessage), 1)))  
    print("")
    print("{:>40}{:>9}".format("Day with the highest number of attacks:", nAttack.index(max(nAttack))+1))
    print("{:>40}{:>9}".format("Day with the highest number of trades:", nTrade.index(max(nTrade))+1))
    print("{:>40}{:>9}".format("Day with the highest number of messages:", nMessage.index(max(nMessage))+1)) 
    print("")

### Trickster detection tool 

In [207]:
def printRealSuspected(allSuspects, nDays):
    
    beneficiaries = dict()
    suspectsSummary = dict()
    finalSuspects = dict()
    
    for day in allSuspects:
        for suspect in allSuspects[day]:
            # print("Suspected: ", suspect)
            if str(suspect) in suspectsSummary:
                suspectsSummary[suspect][0] += 1
            else:
                suspectsSummary[suspect] = list([1, dict(), 0, 0])
                suspectsSummary[suspect][0] = 1
                # print(suspectsSummary[suspect][0])
            # print("TIME: ", suspectsSummary[suspect])
            
            for beneficiary in allSuspects[day][suspect]:
                # print("Beneficiary is: ", beneficiary)
                # print("Benefit is: ", allSuspects[day][suspect][beneficiary])
                # print(str(beneficiary) in suspectsSummary[suspect][1])
                if str(beneficiary) in suspectsSummary[suspect][1]:
                    suspectsSummary[suspect][1][beneficiary][0] += 1
                    suspectsSummary[suspect][1][beneficiary][1] += allSuspects[day][suspect][beneficiary]
                else:
                    suspectsSummary[suspect][1][beneficiary] = [1, allSuspects[day][suspect][beneficiary]]

                    
            # print(suspectsSummary)

    for day in allSuspects:
        for suspect in allSuspects[day]:
            for beneficiary in allSuspects[day][suspect]:
                if beneficiary in suspectsSummary:
                    # print("HERE MAN")
                    suspectsSummary[beneficiary][2] += 1
                    suspectsSummary[beneficiary][3] += allSuspects[day][suspect][beneficiary]
                    # print(suspectsSummary[beneficiary])
                    
            # print(suspectsSummary)
            # print("")
                
    # suspectsSummary = sorted(suspectsSummary.items(), key=lambda e: e[1][2])
    suspectsSummary = OrderedDict(sorted(suspectsSummary.items(), key=lambda t: t[1][0], reverse=True))
    
    print("Complete: ", len(suspectsSummary))
    for suspect in suspectsSummary:
        print(suspect, suspectsSummary[suspect])
        
    for suspect in suspectsSummary:
        if suspectsSummary[suspect][0] > 1 and suspectsSummary[suspect][0] != suspectsSummary[suspect][2]:
            finalSuspects[suspect] = suspectsSummary[suspect]
            
    # print("With elimination: ", len(finalSuspects))
    # for suspect in finalSuspects:
        # print(suspect, finalSuspects[suspect])
    
    # print("")
    # print(finalSuspects)
    
    return finalSuspects
                    
            

def tricksterDetectionTool(totalNet):
    zeroIndegreeVertex = list()

    for vertex in totalNet.vs:
        if vertex.indegree() == 0:
            zeroIndegreeVertex.append(vertex)
    
    '''for vertex in zeroIndegreeVertex:
        print(vertex)
        print(vertex.indegree())
        print(vertex.outdegree())'''
    #print(zeroIndegreeVertex)
    
    suspecteds = dict()
    beneficiary = dict()
    key = ""
    
    for vertex in zeroIndegreeVertex:
        nEdges = 0
        nTrades = 0
        key = ""
        beneficiary = dict()
        for edge in totalNet.es:
            if totalNet.vs[edge.tuple[0]]['id'] == vertex['id']:
                nEdges += 1
                if edge['Label'] == 'Trade':
                    # print(edge)
                    # print(totalNet.vs[edge.tuple[0]]['id'])
                    # print(vertex['id'])
                    if totalNet.vs[edge.tuple[1]]['id'] in beneficiary:
                        beneficiary[totalNet.vs[edge.tuple[1]]['id']] += 1
                    else:
                        beneficiary[totalNet.vs[edge.tuple[1]]['id']] = 1
                    nTrades += 1
        if nTrades/nEdges >= 0.95:
            # print(nEdges)
            # print(nTrades)
            key = str(vertex['id'])
            # print(key)
            # print(beneficiary)
            suspecteds[key] = beneficiary
    
    for key in suspecteds:
        print(key, suspecteds[key])
    return suspecteds

def edgesCount(finalSuspects, allNet):
    edgesInCount = dict()
    edgesOutCount = dict()
    edgesOutPlayer = dict()
    
    activityDay = list()
    
    
    for suspect in finalSuspects:
        
        activityDay.clear()
        edgesInCount.clear()
        edgesOutCount.clear()
        edgesOutPlayer.clear()
        
        for day in allNet:
            for vertex in allNet[day][4].vs:
                if str(vertex['id']) == suspect:
                    activityDay.append(str(day+1))
            


            for edge in allNet[day][4].es:
                if allNetwork[day][4].vs[edge.tuple[0]]['id'] == suspect:
                    if edge['Label'] in edgesOutCount:
                        edgesOutCount[edge['Label']] += 1
                    else:
                        edgesOutCount[edge['Label']] = 1
                    if edge['Label'] == 'Trade':
                        if allNetwork[day][4].vs[edge.tuple[1]]['id'] in edgesOutPlayer:
                            edgesOutPlayer[allNetwork[day][4].vs[edge.tuple[1]]['id']] += 1
                        else:
                            edgesOutPlayer[allNetwork[day][4].vs[edge.tuple[1]]['id']] = 1
                if allNet[day][4].vs[edge.tuple[1]]['id'] == suspect:
                    if edge['Label'] in edgesInCount:
                        edgesInCount[edge['Label']] += 1
                    else:
                        edgesInCount[edge['Label']] = 1
        
        print("SUSPECT: ", suspect)
        print("EDGES IN: ", edgesInCount)
        print("EDGES OUT: ", edgesOutCount)
        print("PLAYER SERVED", edgesOutPlayer)
        print("ACTIVITY DAY: ", activityDay)
        print("")
    
    

## Communities Detection

### TODO
TODO

In [70]:
def startCommunitiesDetection(allianceNet):
    getLouvainCommunities(allianceNet)
    getEigenvectorCommunities(allianceNet)
    getOptiModulaCommunities(allianceNet)
    

def getLouvainCommunities(allianceNet):
    print("TODO")

def getEigenvectorCommunities(allianceNet):
    print("TODO")

def getOptiModulaCommunities(allianceNet):
    print("TODO")  

### TODO
TODO

In [71]:
def VisualizeNetwork(allianceNet):
    print("USING Plotly Python Open Source Graphing Library")

## Main
From cell below we let's start all our analysis

In [208]:
#      TRY, IF YOU WANT!    #
 ###########################
#userNumberTrend(allNetwork)#
#edgeNumberTrend(allNetwork)#
 ########################### 


'''dailySuspicions = dict()
dailySuspicions.clear()
for i in range(0, len(allNetwork)):
    print("DAY: ", str(i+1))
    dailySuspicions[i] = tricksterDetectionTool(allNetwork[i][4])

    # print(dailySuspicions)
    print("")'''

'''for day in dailySuspicions:
    print("DAY: ", int(day+1))
    for suspect in dailySuspicions[day]:
        print(suspect, dailySuspicions[day][suspect])
    print("")'''

edgesCount(printRealSuspected(dailySuspicions, len(dailySuspicions)), allNetwork)

Complete:  233
11407 [23, {'11388': [23, 54], '11106': [6, 12], '5368': [1, 2]}, 0, 0]
7855 [18, {'3778': [18, 22]}, 0, 0]
2789 [15, {'1451': [15, 86]}, 0, 0]
12044 [8, {'4611': [8, 40]}, 0, 0]
9492 [8, {'9861': [8, 11]}, 0, 0]
7118 [6, {'6743': [6, 42]}, 0, 0]
7857 [6, {'3778': [6, 9]}, 0, 0]
11961 [6, {'12073': [6, 13], '12071': [2, 2]}, 0, 0]
10279 [6, {'3282': [6, 6]}, 0, 0]
1306 [6, {'1480': [5, 8], '2157': [1, 1]}, 0, 0]
4034 [6, {'5184': [6, 6]}, 0, 0]
12010 [6, {'6346': [5, 6], '12009': [2, 2], '883': [1, 1]}, 0, 0]
7023 [6, {'1952': [6, 10]}, 0, 0]
11604 [6, {'11991': [6, 23]}, 0, 0]
8041 [5, {'5332': [5, 12]}, 0, 0]
5817 [5, {'9522': [5, 6]}, 0, 0]
10651 [5, {'10649': [5, 62]}, 0, 0]
1420 [5, {'1451': [5, 35]}, 0, 0]
183 [5, {'922': [5, 51], '4323': [1, 1]}, 0, 0]
12069 [5, {'11754': [5, 12]}, 0, 0]
3801 [4, {'5019': [4, 13]}, 0, 0]
7741 [4, {'2119': [3, 17], '7484': [1, 2], '12337': [1, 1]}, 0, 0]
10000 [4, {'107': [4, 21]}, 0, 0]
9671 [4, {'9664': [1, 7], '4666': [3, 4]}, 0

SUSPECT:  11407
EDGES IN:  {'Message': 3, 'Attack': 1, 'Trade': 11}
EDGES OUT:  {'Trade': 78, 'Message': 4}
PLAYER SERVED {'11388': 61, '11106': 12, '9018': 1, '5368': 4}
ACTIVITY DAY:  ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '25', '26', '27', '28', '29', '30']

SUSPECT:  7855
EDGES IN:  {'Attack': 3}
EDGES OUT:  {'Trade': 22}
PLAYER SERVED {'3778': 22}
ACTIVITY DAY:  ['1', '3', '6', '7', '8', '10', '11', '14', '15', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28']

SUSPECT:  2789
EDGES IN:  {'Attack': 13}
EDGES OUT:  {'Trade': 114}
PLAYER SERVED {'1451': 114}
ACTIVITY DAY:  ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '24', '25', '26', '28']

SUSPECT:  12044
EDGES IN:  {'Trade': 20, 'Attack': 7}
EDGES OUT:  {'Message': 2, 'Trade': 76}
PLAYER SERVED {'4611': 76}
ACTIVITY DAY:  ['1', '2', '3', '4', '5', 

SUSPECT:  11745
EDGES IN:  {}
EDGES OUT:  {'Trade': 3}
PLAYER SERVED {'10251': 3}
ACTIVITY DAY:  ['1', '2', '5']

SUSPECT:  11661
EDGES IN:  {'Attack': 379}
EDGES OUT:  {'Trade': 19}
PLAYER SERVED {'11381': 11, '11636': 5, '12140': 2, '1': 1}
ACTIVITY DAY:  ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30']

SUSPECT:  7551
EDGES IN:  {'Message': 40, 'Trade': 38}
EDGES OUT:  {'Trade': 103, 'Message': 7, 'Attack': 11}
PLAYER SERVED {'1192': 3, '1531': 1, '407': 21, '889': 57, '9074': 2, '504': 2, '1225': 1, '10122': 4, '2071': 2, '10418': 2, '3190': 1, '6094': 2, '5859': 3, '510': 2}
ACTIVITY DAY:  ['1', '2', '3', '4', '5', '6', '7', '9', '10', '11', '12', '13', '14', '15', '16', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30']

SUSPECT:  11744
EDGES IN:  {'Attack': 3}
EDGES OUT:  {'Trade': 10}
PLAYER SERVED {'10251': 10}
ACTIVITY DAY:  ['1

SUSPECT:  2539
EDGES IN:  {'Attack': 6, 'Message': 42}
EDGES OUT:  {'Message': 21, 'Trade': 4}
PLAYER SERVED {'1423': 2, '4480': 2}
ACTIVITY DAY:  ['1', '2', '3', '4', '5', '6', '7', '8', '9', '11', '12', '13', '15', '16', '17', '18', '21', '22', '24', '28', '29', '30']

SUSPECT:  7865
EDGES IN:  {'Message': 74, 'Trade': 4}
EDGES OUT:  {'Message': 1, 'Trade': 15}
PLAYER SERVED {'7095': 11, '7672': 1, '1583': 1, '4016': 2}
ACTIVITY DAY:  ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '13', '14', '16', '17', '18', '19', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30']

SUSPECT:  9408
EDGES IN:  {'Trade': 51, 'Message': 27, 'Attack': 4}
EDGES OUT:  {'Trade': 58, 'Attack': 8, 'Message': 36}
PLAYER SERVED {'8058': 11, '2748': 2, '9221': 2, '8932': 1, '5922': 2, '9102': 2, '6346': 12, '6102': 8, '11388': 1, '5368': 2, '6421': 3, '7682': 3, '760': 1, '9293': 1, '1476': 1, '4622': 3, '12494': 1, '8139': 2}
ACTIVITY DAY:  ['1', '2', '3', '4', '6', '7', '8', '9', '10', '11'

SUSPECT:  9837
EDGES IN:  {'Message': 105, 'Trade': 10}
EDGES OUT:  {'Trade': 14, 'Message': 17}
PLAYER SERVED {'2129': 1, '763': 1, '9018': 12}
ACTIVITY DAY:  ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '13', '14', '15', '16', '17', '18', '19', '20', '21', '24', '25', '26', '27', '29', '30']

SUSPECT:  2388
EDGES IN:  {'Message': 37, 'Attack': 50, 'Trade': 2}
EDGES OUT:  {'Trade': 6, 'Message': 3}
PLAYER SERVED {'7138': 2, '10063': 1, '10935': 1, '8574': 2}
ACTIVITY DAY:  ['3', '5', '6', '8', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '27', '29']

SUSPECT:  7718
EDGES IN:  {'Message': 8}
EDGES OUT:  {'Message': 4, 'Trade': 2}
PLAYER SERVED {'8657': 2}
ACTIVITY DAY:  ['4', '10', '13', '14', '18', '19', '21', '27', '28']

SUSPECT:  10451
EDGES IN:  {'Attack': 214, 'Trade': 8}
EDGES OUT:  {'Trade': 48}
PLAYER SERVED {'9261': 18, '2815': 8, '10610': 3, '8017': 6, '345': 11, '322': 1, '10453': 1}
ACTIVITY DAY:  ['1', '2', '3', '4

# I prefer keep a bin cell rather than cry because i need already written code but deleted....
Of course in the final release this will be delete

In [ ]:
# for libraries install
'''!sudo apt install build-essential python-dev libxml2 libxml2-dev zlib1g-dev
!sudo pip install cairocffi
!pip install python-igraph
!pip install louvain'''

attackGraph = ig.Graph()
messageGraph = ig.Graph()
tradeGraph = ig.Graph()

totalPresent = False
alliancePresent = False

situa = 0

graphs = dict()
dayGraphs = list()

nGraph = 30

for i in range(0, 1):
    totalGraph = ig.Graph()
    
    messageGraph = messageGraph.Read_GraphML("../Dataset/Graphml-File/Messages-Network-Graphml/messages-timestamped-2009-12-"+str(i+1)+".graphml")
    for messageVertex in messageGraph.vs:
        if messageVertex.degree() != 0:
            allianceGraph.add_vertices(messageVertex['id'])
        totalPresent = False
        for totalVertex in totalGraph.vs:
            if messageVertex['id'] == totalVertex['name']:
                totalPresent = True
                break
        if not totalPresent:
            if messageVertex.degree() != 0:
                totalGraph.add_vertices(messageVertex['id'])
        totalPresent = False
    
    print("MESSAGE EDGE")        
    for edge in messageGraph.es:
        # edge['edgelabel'] = "Message"
        # print(situa, end = "")
        # print(", ", end = "")
        # situa += 1
        totalGraph.add_edge(messageGraph.vs[edge.tuple[0]], messageGraph.vs[edge.tuple[1]], edgelabel='Message')
        # allianceGraph.add_edge(edge.tuple[0], edge.tuple[1], edgelabel='Message')
    # situa = 0
    # print("")
        

    

    print("MESSAGE GRAPH:", messageGraph.summary())
    print("TOTAL GRAPH:", totalGraph.summary())
    
    print("")
    
    for vertex in totalGraph.vs:
    if vertex['name'] == '9152':
        print(vertex.outdegree())
attackGraph = ig.Graph()
messageGraph = ig.Graph()
tradeGraph = ig.Graph()

totalPresent = False
alliancePresent = False

situa = 0

graphs = dict()
dayGraphs = list()

nGraph = 30

for i in range(0, nGraph):
    totalGraph = ig.Graph()
    allianceGraph = ig.Graph()
    print("Graph number: ", (i + 1))
    
    attackGraph = attackGraph.Read_GraphML("../Dataset/Graphml-File/Attacks-Network-Graphml/attacks-timestamped-2009-12-"+str(i+1)+".graphml")
    for vertex in attackGraph.vs:
        totalGraph.add_vertices(vertex['id'])
        # print(vertex['id'] +", ", end="")
    # print("")
    print("ATTACK EDGE")
    for edge in attackGraph.es:
        # edge['edgelabel'] = "Attack"
        # print(situa, end = "")
        # print(", ", end = "")
        # situa += 1
        totalGraph.add_edge(attackGraph.vs[edge.tuple[0]], attackGraph.vs[edge.tuple[1]], edgelabel='Attack') 
    situa = 0
    # print("")
    
    messageGraph = messageGraph.Read_GraphML("../Dataset/Graphml-File/Messages-Network-Graphml/messages-timestamped-2009-12-"+str(i+1)+".graphml")
    for messageVertex in messageGraph.vs:
        allianceGraph.add_vertices(messageVertex['id'])
        totalPresent = False
        for totalVertex in totalGraph.vs:
            if messageVertex['id'] == totalVertex['name']:
                totalPresent = True
                break
        if not totalPresent:
            totalGraph.add_vertices(messageVertex['id'])
        totalPresent = False
    
    print("MESSAGE EDGE")        
    for edge in messageGraph.es:
        # edge['edgelabel'] = "Message"
        # print(situa, end = "")
        # print(", ", end = "")
        # situa += 1
        totalGraph.add_edge(messageGraph.vs[edge.tuple[0]], messageGraph.vs[edge.tuple[1]], edgelabel='Message')
        allianceGraph.add_edge(messageGraph.vs[edge.tuple[0]], messageGraph.vs[edge.tuple[1]], edgelabel='Message')
    # situa = 0
    # print("")
        
    tradeGraph = tradeGraph.Read_GraphML("../Dataset/Graphml-File/Trades-Network-Graphml/trades-timestamped-2009-12-"+str(i+1)+".graphml")
    for tradeVertex in tradeGraph.vs:
        totalPresent = False
        for totalVertex in totalGraph.vs:
            if tradeVertex['id'] == totalVertex['name']:
                totalPresent = True
                break
        if not totalPresent:
            totalGraph.add_vertices(tradeVertex['id'])
        
    for tradeVertex in tradeGraph.vs:
        alliancePresent = False
        for allianceVertex in allianceGraph.vs:
            if tradeVertex['id'] == allianceVertex['name']:
                alliancePresent = True
                break
        if not alliancePresent:
            allianceGraph.add_vertices(tradeVertex['id'])
        
    
    print("TRADE EDGE")
    for edge in tradeGraph.es:
        # edge['edgelabel'] = "Trade"
        # print(situa, end = "")
        # print(", ", end = "")
        # situa += 1
        totalGraph.add_edge(tradeGraph.vs[edge.tuple[0]], tradeGraph.vs[edge.tuple[1]], edgelabel='Trade')
        allianceGraph.add_edge(tradeGraph.vs[edge.tuple[0]], tradeGraph.vs[edge.tuple[1]], edgelabel='Trade')
    situa = 0
    # allianceGraph = messageGraph.union(tradeGraph)
    # totalGraph = allianceGraph.union(attackGraph)
    # print("")
    
    print("ATTACK GRAPH:", attackGraph.summary())
    print("MESSAGE GRAPH:", messageGraph.summary())
    print("TRADE GRAPH:", tradeGraph.summary())
    print("ALLIANCE GRAPH:", allianceGraph.summary())
    print("TOTAL GRAPH:", totalGraph.summary())
    totalGraph.save("../Dataset/Graphml-File/Total-Network-Graphml/total-2009-12-"+str(i+1)+".graphml")
    allianceGraph.save("../Dataset/Graphml-File/Alliance-Network-Graphml/alliance-2009-12-"+str(i+1)+".graphml")
    print("")
    






'''G = ig.Graph()
G = G.Read_GraphML("../Dataset/Graphml-File/Trades-Network-Graphml/trades-timestamped-2009-12-1.graphml") 
G += G.Read_GraphML("../Dataset/Graphml-File/Messages-Network-Graphml/messages-timestamped-2009-12-1.graphml") 
G += G.Read_GraphML("../Dataset/Graphml-File/Attacks-Network-Graphml/attacks-timestamped-2009-12-1.graphml") 
print(G)'''


# print(G.vs.attribute_names())
# print(G.es.attribute_names())
# print(G.degree())
# print(G.indegree())
# print(G.outdegree())

# totalGraph = totalGraph.Read_GraphML("../Dataset/Graphml-File/Total-Network-Graphml/total-2009-12-1.graphml")
allianceGraph = ig.Graph()
allianceGraph = allianceGraph.Read_GraphML("../Dataset/Graphml-File/Alliance-Network-Graphml/alliance-2009-12-1.graphml")

totalGraph = ig.Graph()
totalGraph = totalGraph.Read_GraphML("../Dataset/Graphml-File/Total-Network-Graphml/total-2009-12-1.graphml")

attackGraph = ig.Graph()
attackGraph = attackGraph.Read_GraphML("../Dataset/Graphml-File/Attacks-Network-Graphml/attacks-timestamped-2009-12-1.graphml")

messageGraph = ig.Graph()
messageGraph = messageGraph.Read_GraphML("../Dataset/Graphml-File/Messages-Network-Graphml/messages-timestamped-2009-12-1.graphml") 

tradeGraph = ig.Graph()
tradeGraph = tradeGraph.Read_GraphML("../Dataset/Graphml-File/Trades-Network-Graphml/trades-timestamped-2009-12-1.graphml")

# print(totalGraph.degree())
# print(messageGraph.degree())
# print(attackGraph.degree())
# print(tradeGraph.degree())
print("AllianceGraph vertex with 0 degree")
for vertex in allianceGraph.vs:
    if vertex.degree() == 0:
        print(vertex['id'] +", ", end = "")
print("")
print("")
print("TotalGraph vertex with 0 degree")
for vertex in totalGraph.vs:
    if vertex.degree() == 0:
        print(vertex['id']+", ", end="")
print("")
print("")

for vertex in attackGraph.vs:
    if vertex.degree() == 0:
        print(vertex['id']+", ", end="")

print("")
print("")
ok = False
print("Attack Graph Check in Total Graph")
for vertex1 in attackGraph.vs:
    ok = False
    if vertex1['id'] == 'n2807':
        print("i am here 1")
    for vertex2 in totalGraph.vs:
        if vertex1['id'] == vertex2['name']:
            ok = True
            break
    if not ok:
        print("error")

print("")
print("")
ok = False
print("Message Graph Check in Total Graph")
for vertex1 in messageGraph.vs:
    ok = False
    if vertex1['id'] == 'n2807':
        print("i am here 2")
    for vertex2 in totalGraph.vs:
        if vertex1['id'] == vertex2['name']:
            ok = True
            break
    if not ok:
        print("error")
        
print("")
print("")
ok = False
print("Trade Graph Check in Total Graph")
for vertex1 in tradeGraph.vs:
    if vertex1['id'] == 'n2807':
        print("i am here 3")
    ok = False
    for vertex2 in totalGraph.vs:
        if vertex1['id'] == vertex2['name']:
            ok = True
            break
    if not ok:
        print("error")

print("")
print("")
ok = False
print("Message Graph Check in Alliance Graph")
for vertex1 in messageGraph.vs:
    ok = False
    for vertex2 in allianceGraph.vs:
        if vertex1['id'] == vertex2['name']:
            ok = True
            break
    if not ok:
        print("error")
        
print("")
print("")
ok = False
print("Trade Graph Check in Alliance Graph")
for vertex1 in tradeGraph.vs:
    ok = False
    for vertex2 in allianceGraph.vs:
        if vertex1['id'] == vertex2['name']:
            ok = True
            break
    if not ok:
        print("error")
    
print(allianceGraph.summary())
print(totalGraph.summary())

for vertex in totalGraph.vs:
    if vertex['id'] == 'n2807':
        print(vertex)
print("")
print("show in the graph")
for vertex in totalGraph.vs:
    if vertex['name'] == '9152':
        print(vertex)
    
print("In Total Graph")
for edge in totalGraph.es:
    if edge.tuple[0] == '9152' or edge.tuple[1] == '9152':
        print(edge)

print("In Message Graph")
for edge in messageGraph.es:
    if edge.tuple[0] == '9152':
        print(edge)
    if edge.tuple[1] == '9152':
        print(edge)
print("")
for vertex in attackGraph.vs:
    if vertex['id'] == '9152':
        print("attack")
        print(vertex)

for vertex in tradeGraph.vs:
    if vertex['id'] == '9152':
        print("trade")
        print(vertex)


for vertex in messageGraph.vs:
    if vertex['id'] == '9152':
        print("message")
        print(vertex)

        
list(attackGraph.vs)

import networkx as nx

messageGraph = nx.MultiDiGraph()
tradeGraph = nx.MultiDiGraph()
attackGraph = nx.MultiDiGraph()
allianceGraph = nx.MultiDiGraph()
totalGraph = nx.MultiDiGraph()

realMessage = nx.MultiDiGraph()
realTrade = nx.MultiDiGraph()
realAttack = nx.MultiDiGraph()

messageGraph = nx.read_graphml("../Dataset/Graphml-File/Messages-Network-Graphml/messages-timestamped-2009-12-1.graphml")
tradeGraph = nx.read_graphml("../Dataset/Graphml-File/Trades-Network-Graphml/trades-timestamped-2009-12-1.graphml")
attackGraph = nx.read_graphml("../Dataset/Graphml-File/Attacks-Network-Graphml/attacks-timestamped-2009-12-1.graphml")
# nx.info(messageGraph)

# print(messageGraph.nodes()[0])

# for edge in messageGraph.edges():   
#    messageGraph[edge[0]][edge[1]].update(label= "Message")

# nx.set_edge_attributes(messageGraph, "Label", "Message")
# nx.set_edge_attributes(tradeGraph, "Label", "Trade")
# nx.set_edge_attributes(attackGraph, "Label", "Attack")

# allianceGraph = nx.disjoint_union(messageGraph, tradeGraph)

# nx.set_edge_attributes(G, bb, 'betweenness')

# for itr in allianceGraph.edges_iter(None, True, True):
#        print(itr)



messageGraph.edges(data=True,keys=True)

for itr in messageGraph.edges_iter(None, True, True):
    realMessage.add_edge(int(itr[0]),int(itr[1]), key=str(itr[2])+"m", Label= "Message")
    
for itr in tradeGraph.edges_iter(None, True, True):
    realTrade.add_edge(int(itr[0]),int(itr[1]), key=str(itr[2])+"t", Label= "Trade")

for itr in attackGraph.edges_iter(None, True, True):
    realAttack.add_edge(int(itr[0]),int(itr[1]), key=str(itr[2])+"a", Label= "Attack")
    

allianceGraph = nx.compose(realMessage, realTrade)
totalGraph = nx.compose(allianceGraph, realAttack)
print("MESSAGE GRAPH: ", nx.info(messageGraph))
print("")
print("TRADE GRAPH: ", nx.info(tradeGraph))
print("")
print("ATTACK GRAPH: ", nx.info(attackGraph))
print("")
print("ALLIANCE GRAPH: ", nx.info(allianceGraph))
print("")
print("TOTAL GRAPH: ", nx.info(totalGraph))

# print(allianceGraph.degree())
print("")
print("")
# print(totalGraph.degree())

nx.write_graphml(realMessage, "../Dataset/Graphml-File/Total-Network-Graphml/total-2009-12-30.graphml")

messageGraph = ig.Graph()
messageGraph = messageGraph.Read_GraphML("../Dataset/Graphml-File/Total-Network-Graphml/total-2009-12-30.graphml") 

print(messageGraph.summary())

# for edge in messageGraph.es:
#    print(edge)
    
#for vertex in messageGraph.vs:
#    print(vertex)

cont = 0
#print(allNetwork[0][4].indegree())
zeroIndegreeVertex = list()

for vertex in allNetwork[0][4].vs:
    if vertex.indegree() == 0:
        zeroIndegreeVertex.append(vertex)

# print(zeroIndegreeVertex)
suspected = dict()
key = ""
nEdges = 0
nTrades = 0

print("{:>7}{:>7}".format("From", "To"))
for vertex in zeroIndegreeVertex:
    nEdges = 0
    nTrades = 0
    for edge in allNetwork[0][4].es:
        if allNetwork[0][4].vs[edge.tuple[0]] == vertex:
            if edge['Label'] == 'Trade':
                nTrades += 1
            nEdges += 1
    if nEdges == nTrades:
        key = "from:" + allNetwork[0][4].vs[edge.tuple[0]]['id'] +" to:"+ allNetwork[0][4].vs[edge.tuple[1]]['id']
        if key not in suspected:
            suspected[key] = 1
        else:
            suspected[key] += 1

print(suspected)
